In [1]:
import pandas as pd
import numpy as np
import unicodedata as ud
import matplotlib.pyplot as plt
import matplotlib
import math

In [105]:
# Uso todo el ancho de la página
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [125]:
dfs = pd.read_csv('/home/sebastian/Facu/OD/guia03/ufo-sightings/scrubbed.csv', low_memory = False)
dfc = pd.read_csv('/home/sebastian/Facu/OD/guia03/ufo-sightings/complete.csv', low_memory = False,\
                  error_bad_lines = False, warn_bad_lines = False)

In [196]:
dfs.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,fecha,chevron
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,1949-10-10 20:30:00,0
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,1949-10-10 21:00:00,0
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,1955-10-10 17:00:00,0
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,1956-10-10 21:00:00,0
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,1960-10-10 20:00:00,0


In [136]:
dfs['fecha'] = pd.to_datetime(dfs['datetime'], errors = 'coerce')
dfc['fecha'] = pd.to_datetime(dfc['datetime'], errors = 'coerce')

In [110]:
#df.info(memory_usage = 'deep')

In [137]:
print('scrubbed: %s' %(dfs.columns))
print('complete: %s' %(dfc.columns))

scrubbed: Index([u'datetime', u'city', u'state', u'country', u'shape', u'duration (seconds)', u'duration (hours/min)', u'comments', u'date posted', u'latitude', u'longitude ', u'fecha', u'chevron'], dtype='object')
complete: Index([u'datetime', u'city', u'state', u'country', u'shape', u'duration (seconds)', u'duration (hours/min)', u'comments', u'date posted', u'latitude', u'longitude', u'fecha', u'chevron'], dtype='object')


### Existen 143 registros de avistajes en Canadá durante el año 2010

In [144]:
mask = (dfs['fecha'] >= pd.Timestamp('2010-01-01')) & (dfs['fecha'] <= pd.Timestamp('2010-12-31'))
dfaux = dfs.loc[mask]
dfaux = dfaux[dfaux['country'] == 'ca']
print('scrubbed: %d' %(len(dfaux)))

mask = (dfc['fecha'] >= pd.Timestamp('2010-01-01')) & (dfc['fecha'] <= pd.Timestamp('2010-12-31'))
dfaux = dfc.loc[mask]
dfaux = dfaux[dfaux['country'] == 'ca']
print('complete: %d' %(len(dfaux)))

scrubbed: 137
complete: 153


### Falso

### El 10% de los reportes incluye objetos con forma de triangulo

In [147]:
dfs['shape'].unique()

array(['cylinder', 'light', 'circle', 'sphere', 'disk', 'fireball',
       'unknown', 'oval', 'other', 'cigar', 'rectangle', 'chevron',
       'triangle', 'formation', nan, 'delta', 'changing', 'egg',
       'diamond', 'flash', 'teardrop', 'cone', 'cross', 'pyramid',
       'round', 'crescent', 'flare', 'hexagon', 'dome', 'changed'],
      dtype=object)

In [158]:
total = len(dfs['shape'].dropna())
print('scrubbed, total formas: %d' %(total))

total = len(dfc['shape'].dropna())
print('complete, total formas: %d' %(total))

scrubbed, total formas: 78400
complete, total formas: 85757


In [159]:
largo = len(dfs[dfs['shape'] == 'triangle'])
print('scrubbed, triangulos: %d' %(largo))

largo = len(dfc[dfc['shape'] == 'triangle'])
print('complete, triangulos: %d' %(largo))

scrubbed, triangulos: 7865
complete, triangulos: 8489


### Verdadero

### Entre los avistajes registrados en Washington, DC, las formas mas populares son luz, esfera y bola de fuego.

In [165]:
dfaux = dfs[dfs['state'] == 'wa']
print('scrubbed')
print(dfaux['shape'].value_counts())

dfaux = dfc[dfc['state'] == 'wa']
print('\ncomplete')
print(dfaux['shape'].value_counts())

scrubbed
light        935
fireball     381
triangle     358
circle       347
unknown      318
sphere       289
other        282
disk         210
oval         189
formation    127
changing      92
flash         82
cigar         66
rectangle     65
cylinder      56
diamond       53
chevron       46
egg           38
teardrop      29
cone          17
cross          8
delta          1
Name: shape, dtype: int64

complete
light        979
fireball     397
triangle     378
circle       375
unknown      356
other        310
sphere       300
disk         229
oval         199
formation    130
changing      98
flash         88
cigar         72
rectangle     66
cylinder      63
diamond       55
chevron       49
egg           40
teardrop      31
cone          18
cross          9
delta          2
Name: shape, dtype: int64


### Falso

### Existen 963 ufos reportados con forma de chevron

In [131]:
dfs['chevron'] = (dfs['shape'] == 'chevron').astype(int)
print('scrubbed.chevron count: %d' %(dfs['chevron'].sum()))

dfc['chevron'] = (dfc['shape'] == 'chevron').astype(int)
print('complete.chevron count: %d' %(dfc['chevron'].sum()))

scrubbed.chevron count: 952
complete.chevron count: 1007


### Falso

### El avistaje mas corto ocurrido en Marzo del 2007 ocurrió en la ciudad de Denver

In [170]:
mask = (dfs['fecha'] >= pd.Timestamp('2007-03-01')) & (dfs['fecha'] <= pd.Timestamp('2007-03-31'))
dfaux = dfs.loc[mask, ['datetime', 'city', 'duration (seconds)']]
dfaux = dfaux.sort_values(by=['duration (seconds)'])
print('scrubbed')
print(dfaux.head(3))

mask = (dfc['fecha'] >= pd.Timestamp('2007-03-01')) & (dfc['fecha'] <= pd.Timestamp('2007-03-31'))
dfaux = dfc.loc[mask, ['datetime', 'city', 'duration (seconds)']]
dfaux = dfaux.sort_values(by=['duration (seconds)'])
print('\ncomplete')
print(dfaux.head(3))

scrubbed
              datetime            city duration (seconds)
33033  3/22/2007 22:15       corvallis                  1
34683   3/4/2007 20:00         herndon                  1
33749  3/27/2007 05:37  millstone twp.                1.5

complete
              datetime                         city duration (seconds)
37231  3/26/2007 04:30                       parker                  0
38415   3/4/2007 19:45                  eagle river                  0
38757   3/6/2007 21:00  gila bend/wellton (between)                  0


### Falso

### La forma mas común en los ufos registrados es circular

In [174]:
print('scrubbed')
print(dfs['shape'].value_counts().head(1))

print('\ncomplete')
print(dfc['shape'].value_counts().head(1))

scrubbed
light    16565
Name: shape, dtype: int64

complete
light    17872
Name: shape, dtype: int64


### Falso

### Las Vegas, NV tiene un registro que aparece en el Top10 de avistajes con mayor duración en segundos para el mes de Marzo 2004.

In [191]:
mask = (dfs['fecha'] >= pd.Timestamp('2004-03-01')) & (dfs['fecha'] <= pd.Timestamp('2004-03-31'))
dfaux = dfs.loc[mask]
dfaux = dfaux.sort_values(by=['duration (seconds)'], ascending=False)
dfaux = dfaux.iloc[0:10]
dfaux = dfaux[dfaux['state'] == 'nv']
print('scrubbed, avistamientos en Nevada: %d' %(len(dfaux)))

mask = (dfc['fecha'] >= pd.Timestamp('2004-03-01')) & (dfc['fecha'] <= pd.Timestamp('2004-03-31'))
dfaux = dfc.loc[mask]
dfaux = dfaux.sort_values(by=['duration (seconds)'], ascending=False)
dfaux = dfaux.iloc[0:10]
dfaux = dfaux[dfaux['state'] == 'nv']
print('complete, avistamientos en Nevada: %d' %(len(dfaux)))

scrubbed, avistamientos en Nevada: 0
complete, avistamientos en Nevada: 0


### Falso

### El promedio de duración de los ufos con formas cambiantes es superior al resto.

In [195]:
dfaux = dfs.loc[:, ['shape', 'duration (seconds)']]
dfaux['duration (seconds)'] = pd.to_numeric(dfaux['duration (seconds)'], errors='coerce')
dfaux = dfaux.groupby('shape').agg({'duration (seconds)':['mean']})
print('\scrubbed')
print(dfaux.head(4))

dfaux = dfc.loc[:, ['shape', 'duration (seconds)']]
dfaux['duration (seconds)'] = pd.to_numeric(dfaux['duration (seconds)'], errors='coerce')
dfaux = dfaux.groupby('shape').agg({'duration (seconds)':['mean']})
print('\ncomplete')
print(dfaux.head(4))

\scrubbed
         duration (seconds)
                       mean
shape                      
changed         3600.000000
changing        2079.513405
chevron          460.317174
cigar           1917.106830

complete
         duration (seconds)
                       mean
shape                      
changed         3600.000000
changing        1933.579579
chevron          488.626564
cigar           1805.303414


### Verdadero

### No tengo la afirmación de la siguiente celda

In [55]:
df3 = df.loc[:, ['city', 'duration (seconds)']]
df3['duration (seconds)'] = pd.to_numeric(df['duration (seconds)'], errors='coerce')
df3 = df3.sort_values(by=['duration (seconds)'], ascending=False)
df3.head(20)

,city,duration (seconds)
559,birmingham (uk/england),97836000.0
53384,ottawa (canada),82800000.0
74660,greenbrier,66276000.0
64390,finley,52623200.0
38261,dont know,52623200.0
69215,englewood,52623200.0
52709,somerset (uk/england),25248000.0
71172,virginia beach,10526400.0
6991,oklahoma,10526400.0
30596,menifee,10526400.0
